In [1]:
!python -V

Python 3.12.7


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [34]:
df = pd.read_parquet('./yellow_tripdata_2023-01.parquet')

In [35]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.00,0.97,1.00,N,161,141,2,9.30,1.00,0.50,0.00,0.00,1.00,14.30,2.50,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.00,1.10,1.00,N,43,237,1,7.90,1.00,0.50,4.00,0.00,1.00,16.90,2.50,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.00,2.51,1.00,N,48,238,1,14.90,1.00,0.50,15.00,0.00,1.00,34.90,2.50,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.00,1.90,1.00,N,138,7,1,12.10,7.25,0.50,0.00,0.00,1.00,20.85,0.00,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.00,1.43,1.00,N,107,79,1,11.40,1.00,0.50,3.28,0.00,1.00,19.68,2.50,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.50,3.96,0.00,1.00,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.50,2.64,0.00,1.00,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.50,5.32,0.00,1.00,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.50,4.43,0.00,1.00,26.58,NaN,NaN


In [36]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [37]:
df.duration.std()

42.594351241920904

In [38]:
pd.options.display.float_format = '{:20,.2f}'.format

In [39]:
df.duration.describe()

count           3,066,766.00
mean                   15.67
std                    42.59
min                   -29.20
25%                     7.12
50%                    11.52
75%                    18.30
max                10,029.18
Name: duration, dtype: float64

In [40]:
df.duration.describe(percentiles = [0.95, 0.98, 0.99])

count           3,066,766.00
mean                   15.67
std                    42.59
min                   -29.20
50%                    11.52
95%                    36.47
98%                    48.73
99%                    57.25
max                10,029.18
Name: duration, dtype: float64

In [41]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [42]:
df.duration.describe()

count           3,009,173.00
mean                   14.20
std                     9.94
min                     1.00
25%                     7.22
50%                    11.55
75%                    18.18
max                    60.00
Name: duration, dtype: float64

In [43]:
3009173/3066766*100

98.1220282212598

In [44]:
categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

/var/folders/_h/28b2k0sj2gj2nppkwcgpr40m0000gn/T/ipykernel_85780/21011638.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [45]:
# train_dicts = df[categorical + numerical].to_dict(orient='records')
train_dicts = df[categorical].to_dict(orient='records')

In [66]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [67]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [68]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649261927686161

# val

In [53]:
df_val = read_dataframe('./yellow_tripdata_2023-02.parquet')

In [54]:
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [56]:
categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']

df_val[categorical] = df_val[categorical].astype(str)

In [57]:
# train_dicts = df[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical].to_dict(orient='records')

In [69]:
X_val = dv.transform(val_dicts)

In [70]:
X_val

<2855951x515 sparse matrix of type '<class 'numpy.float64'>'
	with 5711894 stored elements in Compressed Sparse Row format>

In [71]:
target = 'duration'
y_val = df_val[target].values

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.811817957524739